## Maine Senate overview script

This file pulls in individual contributions to all Senate committees as well as independent expenditures to support or oppose the candidates _and_ party-coordinated expenditures to support or oppose the candidates.

The data fuels a dashboard that contains an overview of the race so far. It also provides the information to reconcile itemized contribution data with campaign totals available from the FEC.

For Susan Collins, there is potentially significant variance between these sources and the cause is not yet clear.

*Question*: What to make of the amended status? Test what's getting pulled out by dropping duplicates on transaction ID. Are the wrong records being retained?

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import datadotworld as dw
from pandas.io.json import json_normalize

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_˜

In [249]:
### INDIVIDUAL DONATIONS TO MAINE SENATE CAMPAIGNS ###
## SENATE COMMITTEE SEARCH ##
#Set search for all 2020 (two-year transaction period) Maine Senate candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'office':'S'
            ,'api_key':config.api_key}

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()

SyntaxError: invalid syntax (<ipython-input-249-190328f3bb1c>, line 48)

In [306]:
replace_dict = {
    '<td>':''
    ,'</td>':''
   ,r'\[|\]':''
   ,r'\\n|\\t':''
   ,r"\'":''
}
## Create and publish candidate id - committee id table
lkup = json_normalize(data=r_cands['results'])[['candidate_id','name','party_full']]
lkup2 = json_normalize(data=r_cands['results'],record_path='principal_committees')[['candidate_ids','committee_id','name']].astype(str)
#Eliminate <td> tags
lkup2.replace(replace_dict,regex=True,inplace=True)
lkup = lkup.merge(lkup2,left_on='candidate_id',right_on='candidate_ids')

colnm = {
    'name_x':'candidate_name'
    ,'name_y':'committee_name'
}
lkup.rename(columns=colnm,inplace=True)
lkup.drop(columns='candidate_ids',inplace=True)

#Write out files to data.world
with dw.open_remote_file('darrenfishell/2020-election-repo','candidate_committee_lookup.csv') as w:
    lkup.to_csv(w,index=False)

In [ ]:
## FOR LOOP TO COLLECT CONTRIBUTION RECORDS ##
#Initialize dataframe collector for itemized contribs
cycle='2020'
idfs=[]
udfs=[]
commid=0

#Initialize query dictionary
itemdict = {
    'per_page':'100'
    ,'sort':'contribution_receipt_date'
    ,'api_key':config.api_key
    ,'is_individual':'true'
    ,'two_year_transaction_period':cycle
    ,'last_index':[]
    ,'last_contribution_receipt_date':[]
    ,'committee_id':comm_ids[commid]
}

#Dict for unitemized contributions
unitemdict = {
'api_key':config.api_key
,'cycle':cycle
,'per_page':'100'
,'committee_id':comm_ids[commid]
}

#Page through results for each committee id
for x in comm_ids):
    
    u_r = requests.get('https://api.open.fec.gov/v1/committee/'+comm_ids[commid]+'/totals',params=unitemdict).json()
    udf = json_normalize(u_r['results'])
    udfs.append(udf)
    
    #Get first itemized payload for a candidate
    itemr = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    #Print itemdict to validate
    print(itemdict)
    
    #Last page variables
    while itemr['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        idf = json_normalize(itemr['results'])
        idfs.append(idf)
                
        #Assign last_index and date values, update itemdict
        last_index=pd.to_numeric(itemr['pagination']['last_indexes']['last_index'])
        last_date=itemr['pagination']['last_indexes']['last_contribution_receipt_date']
        #Update dictionary with new indices
        itemdict.update([('last_index',last_index)
                        ,('last_contribution_receipt_date',last_date)])

        #Get next payload with updated dict
        itemr = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    commid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    itemdict.update([('committee_id',comm_ids[commid])
                     ,('last_index',[])
                     ,('last_contribution_receipt_date',[])])
    
    unitemdict.update([('committee_id',comm_ids[commid])])

In [255]:
# Concatenate all dfs
itemdf=pd.concat(idfs,sort=False,ignore_index=True)
itemdf=itemdf.drop_duplicates(subset='transaction_id')
udf=pd.concat(udfs,sort=False,ignore_index=True)
udf=udf.drop_duplicates()

#ITEMIZED DATA CLEANING#
itemdf['contributor_zip'] = itemdf['contributor_zip'].str[:5]

#Create DataFrame with columns to match itemized table
unitemdf=[]
unitemdf = pd.DataFrame(columns=cols)

## Select data for unitemized df ##
unitemdf[['committee.name'
        ,'committee.party_full'
        ,'contribution_receipt_amount'
        ,'contribution_receipt_date'
        ,'fec_election_type_desc']] = udf[['committee_name'
                                        ,'party_full'
                                        ,'individual_unitemized_contributions'
                                        ,'coverage_end_date'
                                        ,'last_report_type_full']]

#Label as unitemized
unitemdf['contributor_name'] = 'Unitemized individual contributions'
unitemdf['entity_type'] = 'IND'

#Union Itemized and Unitemized contributions
ind_df = pd.concat([itemcull,unitemdf],sort=False,ignore_index=True)

In [263]:
## WRITE OUT INDIVIDUAL DONATION FILES ##
#Write full files out to data.world project


#Test if results are longer than current file. If so, write.
results = dw.query('darrenfishell/2020-election-repo', 'SELECT * FROM individual_senate_contributions')
if len(results.dataframe) > len(itemdf):
    with dw.open_remote_file('darrenfishell/2020-election-repo','individual-senate-contributions.csv') as w:
        itemdf.to_csv(w,index=False)

#Tests if contribution sum is greater than old file. If so, write.        
results = dw.query('darrenfishell/2020-election-repo', 'SELECT * FROM senate_financial_summary')
if sum(results.dataframe['receipts']) > len(udf['receipts']):
    with dw.open_remote_file('darrenfishell/2020-election-repo','senate_financial_summary.csv') as w:
        udf.to_csv(w,index=False)
    

# #QA - compare itemized values to totals
# qadf = itemdf.groupby(['committee.name']).agg({'contribution_receipt_amount':['sum']})
# qadf

# totaldf = udf[['committee_name'
#              ,'individual_itemized_contributions'
#              ,'individual_unitemized_contributions']]

# qadf = qadf.merge(totaldf, left_on='committee.name',right_on='committee_name')

In [268]:
### INDEPENDENT EXPENDITURE RETRIEVAL ###
## SENATE CANDIDATE ID SEARCH ##
cand_ids=json_normalize(data=r_cands['results'])['candidate_id'].tolist()

#Declare loop variables
candid=0
iedict = {
    'per_page':'100'
    ,'api_key':config.api_key
    ,'two_year_transaction_period':cycle
    ,'last_index':[]
    ,'last_expenditure_date':[]
    ,'candidate_id':cand_ids[candid]
}
edfs = []

#Page through results for each committee id
for x in range(0,len(cand_ids)-1):
    
    #Get first itemized payload for a candidate
    ier = requests.get('https://api.open.fec.gov/v1/schedules/schedule_e/',params=iedict).json()
    
    #Print itemdict to validate
    print(iedict)
    
    #Last page variables
    while ier['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        edf = json_normalize(ier['results'])
        edfs.append(edf)
                
        #Assign last_index and date values, update itemdict
        last_index=ier['pagination']['last_indexes']['last_index']
        last_date=ier['pagination']['last_indexes']['last_expenditure_date']
        #Update dictionary with new indices
        iedict.update([('last_index',last_index)
                        ,('last_expenditure_date',last_date)])

        #Get next payload with updated dict
        ier = requests.get('https://api.open.fec.gov/v1/schedules/schedule_e/',params=iedict).json()
    
    candid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    iedict.update([('candidate_id',cand_ids[candid])
                    ,('last_index',[])
                    ,('last_expenditure_date',[])])

edf=pd.concat(edfs,sort=False,ignore_index=True)
edf=edf.drop_duplicates(subset='transaction_id')

#Clean up ZIP codes
edf['committee.zip'] = edf['committee.zip'].str[:5]

#Write out files to data.world

results = dw.query('darrenfishell/2020-election-repo', 'SELECT * FROM senate_independent_expenditures')
if len(results.dataframe) > len(edf):
    with dw.open_remote_file('darrenfishell/2020-election-repo','senate-independent-expenditures.csv') as w:
        edf.to_csv(w,index=False)

{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00061'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S6ME00159'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00111'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00087'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00095'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuopr

In [308]:
edf.head()

,action_code,action_code_full,amendment_indicator,amendment_number,back_reference_schedule_name,back_reference_transaction_id,candidate.candidate_id,candidate.idx,candidate.two_year_period,candidate_first_name,candidate_id,candidate_last_name,candidate_middle_name,candidate_name,candidate_office,candidate_office_district,candidate_office_state,candidate_party,candidate_prefix,candidate_suffix,category_code,category_code_full,committee.affiliated_committee_name,committee.candidate_ids,committee.city,committee.committee_id,committee.committee_type,committee.committee_type_full,committee.cycle,committee.cycles,committee.designation,committee.designation_full,committee.filing_frequency,committee.name,committee.organization_type,committee.organization_type_full,committee.party,committee.party_full,committee.state,committee.state_full,committee.street_1,committee.street_2,committee.treasurer_name,committee.zip,committee_id,conduit_committee_city,conduit_committee_id,conduit_committee_name,conduit_committee_state,conduit_committee_street1,conduit_committee_street2,conduit_committee_zip,dissemination_date,election_type,election_type_full,expenditure_amount,expenditure_date,expenditure_description,file_number,filer_first_name,filer_last_name,filer_middle_name,filer_prefix,filer_suffix,filing_form,image_number,independent_sign_date,independent_sign_name,is_notice,line_number,link_id,memo_code,memo_code_full,memo_text,memoed_subtotal,notary_commission_expiration_date,notary_sign_date,notary_sign_name,office_total_ytd,original_sub_id,payee_city,payee_first_name,payee_last_name,payee_middle_name,payee_name,payee_prefix,payee_state,payee_street_1,payee_street_2,payee_suffix,payee_zip,pdf_url,previous_file_number,report_type,report_year,schedule_type,schedule_type_full,sub_id,support_oppose_indicator,transaction_id
0,A,ADD,N,0,None,None,S6ME00159,99268,2020.0,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,ME,REP,None,None,None,None,NONE,[],WASHINGTON,C00573261,Q,PAC - Qualified,2020,"[2016, 2018, 2020]",U,Unauthorized,M,END CITIZENS UNITED,None,None,None,None,DC,District Of Columbia,P.O. BOX 66005,None,"COLEMAN, KIMBERLY",20035,C00573261,None,None,None,None,None,None,None,2019-10-17T00:00:00,P2020,None,5709.0,2019-10-17T00:00:00,TEXT MESSAGING ESTIMATE,1360147.0,KIMBERLY,COLEMAN,None,None,None,F24,201910189165115847,2019-10-18T00:00:00,"COLEMAN, KIMBERLY",True,24,4101820191674584207,X,None,*,True,None,None,None,10708.43,None,SEATTLE,None,None,None,TATANGO,None,WA,2211 ELLIOTT AVE,STE 200,None,981213622,http://docquery.fec.gov/cgi-bin/fecimg/?201910...,1360147.0,48,2019,SE,ITEMIZED INDEPENDENT EXPENDITURES,4101820191674589295,O,500023548
1,A,ADD,N,0,None,None,S6ME00159,99268,2020.0,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,ME,REP,None,None,004,Advertising Expenses,NONE,[],ARLINGTON,C00695619,V,PAC with Non-Contribution Account - Nonqualified,2020,[2020],U,Unauthorized,M,HEALTH JUSTICE FOR ALL,None,None,None,None,VA,Virginia,2200 WILSON BLVD,SUITE 102-554,"FEIGL-DING, ERIC DR.",22201,C00695619,None,C00695619,None,None,None,None,None,2019-10-11T00:00:00,P2020,None,3650.0,2019-10-11T00:00:00,FACEBOOK COMMUNICATION - IN-KIND VALUE (NON-CO...,1356276.0,ERIC,FEIGL-DING,None,DR.,None,F24,201910139163828324,2019-10-13T00:00:00,"FEIGL-DING, ERIC DR.",True,24,4101320191674174132,None,None,None,False,None,None,None,6291.00,None,ARLINGTON,ERIC,FEIGL-DING,None,"FEIGL-DING, ERIC DR.",DR.,VA,2200 WILSON BLVD,SUITE 102-554,None,22201,http://docquery.fec.gov/cgi-bin/fecimg/?201910...,1356276.0,48,2019,SE,ITEMIZED INDEPENDENT EXPENDITURES,4101320191674175195,O,SE.4245
2,A,ADD,N,0,None,None,S6ME00159,99268,2020.0,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,ME,REP,None,None,None,None,DNC SERVICES CORPORATION / DEMOCRATIC NATIONAL...,[],AUGUSTA,C00179408,Y,Party - Qualified,2020,"[1984, 1986, 1988, 1990, 1992, 1994, 1996, 199...",U,Unauthorized,M,MAINE DEMOCRATIC PARTY,None,None,DEM,DEMOCRATIC PARTY,ME,Maine,PO BOX 5258,320 WATER ST 3RD FLOOR,

In [309]:
## COORDINATED POLITICAL SPENDING RETRIEVAL ## 
#Declare loop variables
candid=0
i=1
pdict = {
    'per_page':'100'
    ,'api_key':config.api_key
    ,'two_year_transaction_period':cycle
    ,'page':i
    ,'candidate_id':cand_ids[candid]
}
pdfs = []

#Page through results for each committee id
for x in range(0,len(cand_ids)-1):
    
    #Get first itemized payload for a candidate
    p_r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_f/',params=pdict).json()
    
    #Print itemdict to validate
    print(pdict)
    
    #Last page variables
    while p_r['pagination']['page']<=p_r['pagination']['pages']:
        
        #Store results of payload
        pdf = json_normalize(p_r['results'])
        pdfs.append(pdf)
                
        #Increment and update page
        i+=1
        pdict.update([('page',i)])

        #Get next payload with updated dict
        p_r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_f/',params=pdict).json()
    
    candid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    pdict.update([('candidate_id',cand_ids[candid])
                    ,('page',1)])

pdf=pd.concat(pdfs,sort=False,ignore_index=True)
pdf=pdf.drop_duplicates(subset='transaction_id')
pdf['committee.zip'] = pdf['committee.zip'].str[:5]
pdf

{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'page': 1, 'candidate_id': 'S0ME00061'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'page': 1, 'candidate_id': 'S6ME00159'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'page': 1, 'candidate_id': 'S0ME00111'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'page': 1, 'candidate_id': 'S0ME00087'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'page': 1, 'candidate_id': 'S0ME00095'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'page': 1, 'candidate_id': 'S0ME00053'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', '

,action_code,action_code_full,aggregate_general_election_expenditure,back_reference_schedule_name,back_reference_transaction_id,candidate_first_name,candidate_id,candidate_last_name,candidate_middle_name,candidate_name,candidate_office,candidate_office_district,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_prefix,candidate_suffix,catolog_code,catolog_code_full,committee.affiliated_committee_name,committee.candidate_ids,committee.city,committee.committee_id,committee.committee_type,committee.committee_type_full,committee.cycle,committee.cycles,committee.designation,committee.designation_full,committee.filing_frequency,committee.name,committee.organization_type,committee.organization_type_full,committee.party,committee.party_full,committee.state,committee.state_full,committee.street_1,committee.street_2,committee.treasurer_name,committee.zip,committee_designated_coordinated_expenditure_indicator,committee_id,committee_name,conduit_committee_city,conduit_committee_id,conduit_committee_name,conduit_committee_state,conduit_committee_street1,conduit_committee_street2,conduit_committee_zip,designated_committee_id,designated_committee_name,election_cycle,entity_type,entity_type_desc,expenditure_amount,expenditure_date,expenditure_purpose_full,expenditure_type,expenditure_type_full,file_number,filing_form,image_number,line_number,link_id,load_date,memo_code,memo_code_full,memo_text,original_sub_id,payee_first_name,payee_last_name,payee_middle_name,payee_name,pdf_url,report_type,report_year,schedule_type,schedule_type_full,sub_id,subordinate_committee,subordinate_committee_id,transaction_id,unlimited_spending_flag,unlimited_spending_flag_full,subordinate_committee.affiliated_committee_name,subordinate_committee.candidate_ids,subordinate_committee.city,subordinate_committee.committee_id,subordinate_committee.committee_type,subordinate_committee.committee_type_full,subordinate_committee.cycle,subordinate_committee.cycles,subordinate_committee.designation,subordinate_committee.designation_full,subordinate_committee.filing_frequency,subordinate_committee.name,subordinate_committee.organization_type,subordinate_committee.organization_type_full,subordinate_committee.party,subordinate_committee.party_full,subordinate_committee.state,subordinate_committee.state_full,subordinate_committee.street_1,subordinate_committee.street_2,subordinate_committee.treasurer_name,subordinate_committee.zip
0,N,NO CHANGE,None,None,None,SUSAN M,S6ME00159,0,None,"COLLINS, SUSAN M",S,00,SENATE,ME,MAINE,None,None,None,None,NY SAL 96 TN UN 96 RNC-ST VIC 96,[],WASHINGTON,C00003418,Y,Party - Qualified,1996,"[1976, 1978, 1980, 1982, 1984, 1986, 1988, 199...",U,Unauthorized,M,REPUBLICAN NATIONAL COMMITTEE - RNC,None,None,REP,REPUBLICAN PARTY,DC,District Of Columbia,310 FIRST STREET S.E.,None,WILLIAM J. MCMANUS,20003,None,C00003418,REPUBLICAN NATIONAL COMMITTEE - RNC,None,None,None,None,None,None,None,None,None,1996,None,None,122640,1996-10-03T00:00:00+00:00,COORDINATED EXPENDITURE,24C,COORDINATED EXPENDITURE,NaN,F3X,96031173567,25,3040120021023080717,2017-06-17T16:33:49.237049+00:00,None,None,None,None,None,None,None,None,http://docquery.fec.gov/cgi-bin/fecimg/?960311...,12G,1996,SF,ITEMIZED COORD EXPEND MADE BY POL PTY COMM OR ...,3062020110012932151,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A,ADD,500.00,None,None,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,SENATE,ME,MAINE,None,None,None,None,COLLINS MAINE 2020,[],AUGUSTA,C00003111,Y,Party - Qualified,2020,"[1976, 1978, 1980, 1982, 1984, 1986, 1988, 199...",U,Unauthorized,M,MAINE REPUBLICAN PARTY,None,None,REP,REPUBLICAN PARTY,ME,Maine,9 HIGGINS STREET,None,"LOMBARD, BEN",04330,N,C00003111,None,None,C00003111,None,None,None,None,None,None,None,2020,ORG,ORGANIZATION,500,2019-08-12T00:00:00+00:00,DONOR PROSPECTING,24C,COORDINATED EXPENDITURE,1352039.0,F3X,201909169163325918,25,4091620191672682570,2019-09-17T

In [24]:
#Write out file to data.world
results = dw.query('darrenfishell/2020-election-repo', 'SELECT * FROM senate_party_coordinated_expenditures')

if len(results.dataframe) > len(itemdf):
    with dw.open_remote_file('darrenfishell/2020-election-repo','senate-party-coordinated-expenditures.csv') as w:
        pdf.to_csv(w,index=False)

NameError: name 'itemdf' is not defined


The following command must be run outside of the IPython shell:

    $ pip freeze > requirements.txt

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/
